In [17]:
!pip install requests sqlmodel
from datetime import datetime
import requests,json

from sqlmodel import Session

from models import Athlete, Activity, Owner, Period
from database import create_db_and_tables, get_session, engine

In [20]:
def getData(field='athletes'):
    header = {
        'accept': 'application/json',
        'authorization': f"Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiJ9.eyJhdWQiOiI0NjFiMTExMS02ZjdhLTRkYmItOWQyOS0yMzAzOWZlMjI4OGUiLCJqdGkiOiJhN2U5MjBhN2RhMDJkMjFkMDVhNTljN2E0ZTVmYzAxMDkyMGE0NDRhNTY2Mjg1ZDJkMDk4Y2Y1OWQ5OWEwODhkYmQ0NTVjOGQyNGYxNzkwMyIsImlhdCI6MTczNzY2OTUxMC42MDI1OTUsIm5iZiI6MTczNzY2OTUxMC42MDI1OTcsImV4cCI6NDg5MTI2OTUxMC41OTMyODgsInN1YiI6IjdhMmQ3MDM4LTMxYzgtNDUxNS1iZTM4LTkwNDRiOWY3ZWQ0OSIsInNjb3BlcyI6WyJjb25uZWN0Iiwic2Vuc29yLXJlYWQtb25seSIsImF0aGxldGVzLXVwZGF0ZSIsInRhZ3MtdXBkYXRlIiwiYWN0aXZpdGllcy11cGRhdGUiLCJhbm5vdGF0aW9ucy11cGRhdGUiLCJwYXJhbWV0ZXJzLXVwZGF0ZSJdfQ.vUaUio4sdG1fxs9FgO1DHtSRDsInCUiHdbKh01o8o4AjzIcoKXcg_S0vnJr7yO5TWv5wn5_9wYbY4b3bO-48WDr3lZY4EsTEKNCwQe0PzyJ11JJEMZpqylisp2oM97PnqVZWIDXD1tatWrAq0i0BBX93TCfanvzQf93eVKEzCD89S8UwFW5gyrwTH2Zx0HcXUo7PIOJTc-Ie1NDoItAYrSyIfzkJxHV_vOadiTuHuDyl6DFuyJrPwJMJW4MGhb3l1L88HlYNw297ePeq1HOy8vl1cBgvjjpxqp9lTSGj1nZjUjGFkCT6jqh2DmZyrVr0F-Pi3QSwPIOV3ziYYeyenRLBDZddY50ioLC9kk5Y2tFOKmBT6rFK881ek7RyNwt0i8k-RSVCrcgm1QLnPCk6DeOWUPSquSQY5TrUmSboyCMss6mNml_dyw7Kme5uZhv37H8B82ZIal73q_zxIuoX6wvu-pUinrarCAfyDJi37T72gG1w9EndE5eF-uhHUINtVIRuRVPzNlPvGUsnBIoqsm_6N_jS4tQsDSPewGbzZ-PcX2bzL3j5QcSZuu0zJjCJQ740pjr1wqXVWW_qsPj-Fauq54aGfHTDJZfI0U2dW9E5TeG4-2yw3en9-79_IIVoi0u4fxMzI8FeeD_h1CCrNNDg5ZcuhpXpruucQhpR3fE"
    }

    endpoint = 'https://connect-us.catapultsports.com/api/v6'


    #This string can be changed depending on what data you want.  See the link
    #in the google docs or below for more info on the api.
    #https://docs.connect.catapultsports.com/reference/introduction

    url = f'{endpoint}/{field}'

    try:
        response = requests.get(url,headers=header)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as err:
        print(f"Error: {err}")

In [12]:
# Save the retrieved data to a JSON file locally
data = getData()
print(len(data))
# print(type(data))
# print(data)

with open('athletes_data.json', 'w') as json_file:
    json.dump(json.loads(data), json_file)


3049718


In [19]:
def parse_datetime(date_string):
    if date_string:
        return datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")
    return None

def store_athletes(data):
    create_db_and_tables()
    session = next(get_session())

    for athlete_data in data:
        athlete = Athlete(
            id=athlete_data["id"],
            first_name=athlete_data["first_name"],
            last_name=athlete_data["last_name"],
            gender=athlete_data["gender"],
            jersey=athlete_data["jersey"],
            nickname=athlete_data["nickname"],
            height=athlete_data["height"],
            weight=athlete_data["weight"],
            date_of_birth=datetime.fromtimestamp(athlete_data["date_of_birth"]),
            velocity_max=athlete_data["velocity_max"],
            acceleration_max=athlete_data["acceleration_max"],
            heart_rate_max=athlete_data["heart_rate_max"],
            player_load_max=athlete_data["player_load_max"],
            image=athlete_data["image"],
            icon=athlete_data["icon"],
            stroke_colour=athlete_data["stroke_colour"],
            fill_colour=athlete_data["fill_colour"],
            trail_colour_start=athlete_data["trail_colour_start"],
            trail_colour_end=athlete_data["trail_colour_end"],
            is_synced=athlete_data["is_synced"],
            is_deleted=athlete_data["is_deleted"],
            created_at=parse_datetime(athlete_data["created_at"]),
            modified_at=parse_datetime(athlete_data["modified_at"]),
            date_of_birth_date=athlete_data["date_of_birth_date"],
            is_demo=athlete_data["is_demo"],
            current_team_id=athlete_data["current_team_id"],
            max_player_load_per_minute=athlete_data["max_player_load_per_minute"],
            position=athlete_data["position"],
            position_id=athlete_data["position_id"],
            position_name=athlete_data["position_name"]
        )
        session.add(athlete)

    session.commit()
    session.close()

# Load the JSON data
with open('athletes_data.json', 'r') as f:
    athletes_data = json.load(f)

# Store the athletes
store_athletes(athletes_data)

In [16]:
def parse_datetime(date_string):
    if date_string:
        return datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")
    return None

def store_activities(data):
    create_db_and_tables()

    with Session(engine) as session:
        for activity_data in data:
            # # Create Owner
            # owner = Owner(
            #     id=activity_data["owner"]["id"],
            #     customer_id=activity_data["owner"]["customer_id"],
            #     name=activity_data["owner"]["name"],
            #     email=activity_data["owner"]["email"],
            #     is_synced=activity_data["owner"]["is_synced"],
            #     is_deleted=activity_data["owner"]["is_deleted"],
            #     created_at=parse_datetime(activity_data["owner"]["created_at"]),
            #     modified_at=parse_datetime(activity_data["owner"]["modified_at"]),
            #     default=activity_data["owner"]["default"],
            #     full_name=activity_data["owner"]["full_name"],
            # )
            # session.add(owner)

            # Create Activity
            activity = Activity(
                id=activity_data["id"],
                name=activity_data["name"],
                start_time=activity_data["start_time"],
                end_time=activity_data["end_time"],
                modified_at=parse_datetime(activity_data["modified_at"]),
                game_id=activity_data["game_id"],
                owner_id=activity_data["owner_id"],
                athlete_count=activity_data["athlete_count"],
                period_count=activity_data["period_count"],
                # owner=owner
            )
            session.add(activity)

            # Create Periods
            for period_data in activity_data["periods"]:
                period = Period(
                    id=period_data["id"],
                    activity_id=period_data["activity_id"],
                    period_depth_id=period_data["period_depth_id"],
                    name=period_data["name"],
                    start_time=period_data["start_time"],
                    start_centiseconds=period_data["start_centiseconds"],
                    end_time=period_data["end_time"],
                    end_centiseconds=period_data["end_centiseconds"],
                    lft=period_data["lft"],
                    rgt=period_data["rgt"],
                    is_synced=period_data["is_synced"],
                    is_deleted=period_data["is_deleted"],
                    is_injected=period_data["is_injected"],
                    created_at=parse_datetime(period_data["created_at"]),
                    modified_at=parse_datetime(period_data["modified_at"]),
                    user_id=period_data["user_id"]
                )
                session.add(period)
            #
            # # Create Tags
            # for tag_data in activity_data["tag_list"]:
            #     tag = Tag(
            #         id=tag_data["id"],
            #         tag_type_id=tag_data["tag_type_id"],
            #         name=tag_data["name"],
            #         tag_type_name=tag_data["tag_type_name"],
            #         tag_name=tag_data["tag_name"]
            #     )
            #     session.add(tag)
            #     activity.tag_list.append(tag)

        session.commit()
        session.close()


# Usage
with open('activities_data.json', 'r') as f:
    activities_data = json.load(f)
store_activities(activities_data)


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: activity.id
[SQL: INSERT INTO activity (id, name, start_time, end_time, modified_at, game_id, owner_id, athlete_count, period_count) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [('8680b45a-af2c-4533-8e7c-aa796dfc8749', '1-30-25 Training', 1738260321, 1738268470, '2025-01-30 22:50:37.000000', '8680b45a-af2c-4533-8e7c-aa796dfc8749', '7a2d7038-31c8-4515-be38-9044b9f7ed49', 19, 12), ('aab46b07-cfa6-48f3-a550-383f9accb4d2', '1-28-25 Training', 1738087693, 1738097275, '2025-01-28 21:05:52.000000', 'aab46b07-cfa6-48f3-a550-383f9accb4d2', '7a2d7038-31c8-4515-be38-9044b9f7ed49', 16, 10), ('3024f0ea-c087-464b-97ec-c63ecea7a7d7', '1-27-24 Training', 1738006619, 1738025684, '2025-01-28 21:05:52.000000', '3024f0ea-c087-464b-97ec-c63ecea7a7d7', '7a2d7038-31c8-4515-be38-9044b9f7ed49', 17, 8), ('c2cb2d5e-2849-44c2-adba-08ed7ed124b9', '1-24-25 Training', 1737747132, 1737752165, '2025-01-27 18:21:41.000000', 'c2cb2d5e-2849-44c2-adba

Player running around

115 long by 75 wide (yards)

In [21]:
data = getData('activities/d3a79f2e-9fab-40b3-b64e-56edbb966e05/athletes/16bef5df-a172-430c-ae67-35ee189d6fc1/sensor')
print(len(data))
# print(type(data))
# print(data)

with open('someone_running_around_data.json', 'w') as json_file:
    json.dump(json.loads(data), json_file)

5618750


In [34]:
with open('someone_running_around_data.json', 'r') as f:
    activities_data = json.load(f)
    time_points = activities_data[0]['data']
    # for point in data[0:1]:
    #     print()

    import matplotlib.pyplot as plt
    import matplotlib.animation as animation
    import numpy as np
    from matplotlib.patches import Rectangle

    # Assuming you have your data in a list called 'data'
    # Extract x, y coordinates and timestamps
    x = [point['x'] for point in time_points]
    y = [point['y'] for point in time_points]
    timestamps = [point['ts'] for point in time_points]

    def process_data(data):
        processed_data = []
        for i, point in enumerate(data):
            if i % 10000 == 0:
                processed_data.append({
                    'x': point['x'],
                    'y': point['y'],
                    'ts': point['ts']
                })
        return processed_data

    # Assuming 'data' is your list of 500,000 data points
    processed_data = process_data(time_points)


# Normalize coordinates to fit a standard soccer field
    field_length = 105  # meters
    field_width = 68  # meters
    x_normalized = [(xi - min(processed_data[x])) / (max(processed_data[x]) - min(processed_data[x])) * field_length for xi in processed_data[x]]
    y_normalized = [(yi - min(processed_data[y])) / (max(processed_data[y]) - min(processed_data[y])) * field_width for yi in processed_data[y]]

    # Set up the figure and axis
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim(0, field_length)
    ax.set_ylim(0, field_width)

    # Draw the soccer field
    ax.add_patch(Rectangle((0, 0), field_length, field_width, fill=False, color='green'))
    ax.add_patch(Rectangle((0, 0), 16.5, field_width, fill=False, color='white'))
    ax.add_patch(Rectangle((field_length-16.5, 0), 16.5, field_width, fill=False, color='white'))
    ax.add_patch(Rectangle((field_length/2-9.15, 0), 18.3, field_width, fill=False, color='white'))
    plt.plot([field_length/2, field_length/2], [0, field_width], color='white')
    center_circle = plt.Circle((field_length/2, field_width/2), 9.15, fill=False, color='white')
    ax.add_artist(center_circle)

    # Initialize the plot
    line, = ax.plot([], [], 'r-', linewidth=2, alpha=0.6)
    point, = ax.plot([], [], 'ro', markersize=8)

    # Set title and remove axes
    ax.set_title("Player Movement on Soccer Field", fontsize=16)
    ax.axis('off')

    # Animation update function
    def update(frame):
        line.set_data(x_normalized[:frame], y_normalized[:frame])
        point.set_data(x_normalized[frame], y_normalized[frame])
        return line, point

    # Create the animation
    # anim = animation.FuncAnimation(fig, update, frames=len(x), interval=50, blit=True)

    # Add a colorbar to show time progression
    norm = plt.Normalize(min(processed_data[timestamps]), max(processed_data[timestamps]))
    sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm)
    cbar.set_label('Time', fontsize=12)

    plt.tight_layout()

    # Save the animation (optional)
    # anim.save('player_movement.mp4', writer='ffmpeg', fps=30)

    plt.show()



TypeError: list indices must be integers or slices, not list